In [22]:
from jupyter_plotly_dash import JupyterDash

import base64

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name

import AACModule
from AACModule import AnimalShelter


###########################
# Data Manipulation / Model
###########################
username = "myUserAdmin2"
password = "Alexander9"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.readAll({}))
df = df.iloc[:, 1:] #This is needed to omit the Object ID created by Mongo



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')
image_filename = 'GraziosoSalvareLogo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


                            
app.layout = html.Div([
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),

    html.Div(
        dcc.RadioItems(
            #This creates the radio items style buttons with the labels needed
            id = "filter-type",
            options =[
            {"label": "Water Rescue", "value": "water"},
            {"label": "Wilderness Rescue", "value": "wild"},
            {"label": "Disaster Rescue", "value": "dist"},
            {"label": "Reset", "value": "res"},
            ],
            value = "res",
            labelStyle = {"display": "inline-block"},
            )
    ), 
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
    ),
    html.Br(),
    html.Hr(),
    html.H1('Andrew McPherson'), #Unique HTML component header
    html.A([
            html.Img(
                src='data:image/png;base64,{}'.format(encoded_image.decode()),
                style={
                    'height' : '250px',
                    'width' : '250px',
                })
    ], href='https://www.snhu.edu'), #Added a link to the image
           
           
    
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])
#############################################
# Interaction Between Components / Controller
#############################################
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    if filter_type == "water":
        #Filter for the Water Rescue desired gender, breeds, and age.
        df = pd.DataFrame(list(shelter.read({'$and': [{'sex_upon_outcome': 'Intact Female'},
                                                     {'$or': [
                                                         {'breed': 'Labrador Retriever Mix'},
                                                         {'breed': 'Chesapeake Bay Retriever'},
                                                         {'breed': 'Newfoundland Mix'},
                                                         {'breed': 'Newfoundland/Labrador Retriever'},
                                                         {'breed': 'Labrador Retriever/Newfoundland'},
                                                         {'breed': 'Newfoundland/Australian Cattle Dog'},
                                                         {'breed': 'Newfoundland/Great Pyrenees'}]
                                                     },
                                                     {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 26}},
                                                               {'age_upon_outcome_in_weeks': {'$lte': 156}}]
                                                     }]
                                                })))
        df = df.iloc[:, 1:]
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)
        
    elif filter_type == "wild":
        #Filter for the Wilderness Rescue desired gender, breeds, and age.
        df = pd.DataFrame(list(shelter.read({'$and': [{'sex_upon_outcome': 'Intact Male'},
                                                     {'$or': [
                                                         {'breed': 'German Shepherd'},
                                                         {'breed': 'Alaskan Malamute'},
                                                         {'breed': 'Old English Sheepdog'},
                                                         {'breed': 'Rottweiler'},
                                                         {'breed': 'Siberian Husky'}]
                                                     },
                                                      {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 26}},
                                                                {'age_upon_outcome_in_weeks': {'$lte': 156}}]
                                                     }]
                                                })))
        df = df.iloc[:, 1:]
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)
    elif filter_type == 'dist':
        #Filter for the Disaster Rescue desired gender, breeds, and age.
        df = pd.DataFrame(list(shelter.read({'$and': [{'sex_upon_outcome': 'Intact Male'},
                                                     {'$or': [
                                                         {'breed': 'Doberman Pinscher'},
                                                         {'breed': 'German Shepherd'},
                                                         {'breed': 'Golden Retriever'},
                                                         {'breed': 'BLoodhound'},
                                                         {'breed': 'ROttwiler'}]
                                                     },
                                                      {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 20}},
                                                                {'age_upon_outcome_in_weeks': {'$lte': 300}}]
                                                         
                                                     }]
                                                })))
        df = df.iloc[:, 1:]
        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)
        
    elif filter_type == 'res':
        #Reset filter to normal view
        df = pd.DataFrame.from_records(shelter.readAll({}))
        df = df.iloc[:, 1:]
        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    #Added code to display a pie chart.
    dff = pd.DataFrame.from_dict(viewData)
    names =  dff['breed'].value_counts().keys().tolist()
    values = dff['breed'].value_counts().tolist()
    return [
        dcc.Graph(            
            figure = px.pie(
                data_frame = dff,
                values = values,
                names = names,
                color_discrete_sequence = px.colors.sequential.RdBu,
                width = 800,
                height = 500
            )
        )    
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])

def update_map(viewData):
#Added in the code for the geolocation chart (Still using base code from last week. Struggled here)
    dff = pd.DataFrame.from_dict(viewData)
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ]
    

app